# Assignment #2

# In This assignment you are asked to read a data which include 48505 articles (Documents). Then fint the most similar documents using Locality Sensitive Hashing. Follow the lecture covering this topic step by step.

In [1]:
import pandas as pd
import numpy as np
import json
import nltk
from nltk import ngrams
import string
from scipy.sparse import csr_matrix
import copy

In [2]:
num_frequent_grams = 10000
num_hash_functions = 200
num_buckets = 10000
num_bands = 20

In [3]:
! pip install nltk

Defaulting to user installation because normal site-packages is not writeable


## 1. Data is available in Json format and you need to read it. 'https://www.ux.uis.no/~vsetty/data/assignment2_aricles.json' (5 points)

In [3]:
# 1. Read Data
raw_data = pd.read_json('https://www.ux.uis.no/~vsetty/data/assignment2_aricles.json')
df = raw_data.copy(deep = True)
df.head()

,article_id,Title,Content
0,0,Tikcro enters into research and license agreem...,Tikcro enters into research and license agreem...
1,1,Facebook Friend Request Nearly Cost One North ...,A North Carolina woman is trying to warn other...
2,2,Amlin plc UK Regulatory Announcement: Total Vo...,LONDON--(BUSINESS WIRE)--\n\nAMLIN plc\n\nTOTA...
3,3,Khaleda asks for security,Khaleda asks for security\n\n\n\nBNP Chairpers...
4,4,Liberian Health Clinics Reopen Slowly with Ren...,Liberian Health Clinics Reopen Slowly with Ren...


In [4]:
df = df.drop(index = [i for i in range(15000, 48505)])

In [5]:
df.shape

(15000, 3)

## 2. Shingle the documents (10 points)
### Tips:
* Use string package to cleanup the articles e.g, str.maketrans('', '', string.
punctuation)
* It is better to convert text to lower case that way you get fewer n-grams
* apply ngrams(x.split(), n) using ngrams from nltk on the content + title for computing n-grams, for this data n = 2 is suffcient
  * You can use n-gram at word level for this task
  * try with different n-gram values 
  * You can use ngrams from nltk for this

In [6]:
string.punctuation + "”“’"

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~”“’'

In [7]:
# 2. Shingle the Documents (cleaning)
nltk.download('punkt')
nltk.download('stopwords')

number_of_articles = df.shape[0]
stop_words = set(nltk.corpus.stopwords.words('english'))

for i in ["Title", "Content"]:
    # Convert to Lowercase and replace newline with space
    df[i] = df[i].apply(lambda x: x.lower().replace('\n', ' '))
    # Remove special characters and Numbers
    df[i] = df[i].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation + "”“’0123456789")))
    # Remove stop words so that we don't include them in ngrams
    df[i] = df[i].apply(lambda x: ' '.join([w for w in nltk.tokenize.word_tokenize(x) if w not in stop_words]))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rehman\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rehman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# Calculate ngrams
df["ngrams-2"] = (df["Title"] + df["Content"]).apply(lambda x: list(ngrams(x.split(), 2)))
# df["ngrams-3"] = (df["Title"] + df["Content"]).apply(lambda x: list(ngrams(x.split(), 3)))
# df["ngrams-4"] = (df["Title"] + df["Content"]).apply(lambda x: list(ngrams(x.split(), 4)))

In [9]:
df.head()

,article_id,Title,Content,ngrams-2
0,0,tikcro enters research license agreement yeda,tikcro enters research license agreement yeda ...,"[(tikcro, enters), (enters, research), (resear..."
1,1,facebook friend request nearly cost one north ...,north carolina woman trying warn others facebo...,"[(facebook, friend), (friend, request), (reque..."
2,2,amlin plc uk regulatory announcement total vot...,londonbusiness wire amlin plc total voting rig...,"[(amlin, plc), (plc, uk), (uk, regulatory), (r..."
3,3,khaleda asks security,khaleda asks security bnp chairperson khaleda ...,"[(khaleda, asks), (asks, securitykhaleda), (se..."
4,4,liberian health clinics reopen slowly renewed ...,liberian health clinics reopen slowly renewed ...,"[(liberian, health), (health, clinics), (clini..."


## 3. Convert n-grams into binary vector representation for each document. You can do some optimzations if the matrix is too big. (10 points)
* For example,

  * Select top 10000 most frequent n-grams.
  * You may also try smaller values of n (like 2 or 3) which result in fewer n-grams.
  * Finally, you can also try sparse matrix representation. Like csr_matrix from scipy.sparse. It works even with full vocabulary.
    * Given a list of n-grams for each document, see how to builid a sparse matrix here https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

In [10]:
from collections import Counter

In [11]:
def getFrequentNgrams(articles):
    # Your code Here
    # Select most frequent n-grams
    ngrams = []
    for i in articles["ngrams-2"]:
        for j in i:
            string_ngram = f'{j[0]} {j[1]}'
            ngrams.append(string_ngram)
    return [x[0] for x in Counter(ngrams).most_common(num_frequent_grams)]

In [12]:
# Code from https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html
def getBinaryMatrix(docs):
    indptr = [0]
    indices = []
    data = []
    vocabulary = {}
    for doc in docs:
        for word in doc:
            index = vocabulary.setdefault(word, len(vocabulary))
            indices.append(index)
            data.append(1)
        indptr.append(len(indices))
    return csr_matrix((data, indices, indptr), dtype=int).toarray()

In [13]:
frequent_ngrams = getFrequentNgrams(df)

In [14]:
frequent_ngrams

['new years',
 'new year',
 'new york',
 'united states',
 'years eve',
 'prime minister',
 'per cent',
 'last year',
 'airasia flight',
 'flight qz',
 'associated press',
 'bay area',
 'search rescue',
 'san francisco',
 'let us',
 'years ago',
 'security council',
 'said statement',
 'high school',
 'officials said',
 'bad weather',
 'java sea',
 'next year',
 'first time',
 'last week',
 'family members',
 'air force',
 'email address',
 'human rights',
 'said wednesday',
 'police said',
 'social media',
 'two years',
 'pangkalan bun',
 'local news',
 'across country',
 'united nations',
 'page click',
 'islamic state',
 'three years',
 'oil prices',
 'south korea',
 'law enforcement',
 'posted onby',
 'police officers',
 'supreme court',
 'rescue agency',
 'fairfield county',
 'rights reserved',
 'air traffic',
 'missing airasia',
 'cicsac cicsac',
 'market research',
 'saudi arabia',
 'news agency',
 'years day',
 'return home',
 'around world',
 'airasia plane',
 'hong kong',
 'h

With only the frequent ngrams we only get 827 cols in binary matrix. So we select all the ngrams and remove the ones that are not frequent

In [15]:
# Of all the ngrams of documents remove the ones not in frequent ngrams
docs = df["ngrams-2"].copy(deep = True)
docs = docs.apply(lambda x: [f'{i[0]} {i[1]}' for i in x])
docs = docs.apply(lambda row: np.intersect1d(row, frequent_ngrams))

In [17]:
print(len(docs))
print(docs)

15000
0        [development new, research development, st dec...
1        [abc news, could cost, daily basis, didnt know...
2                            [total number, voting rights]
3        [according news, court hearing, former prime, ...
4        [blood pressure, didnt even, dont know, dozen ...
                               ...                        
14995    [another story, available either, back previou...
14996    [cause death, court appearance, court monday, ...
14997    [backpacks filled, claimed lives, cold tempera...
14998                                   [associated press]
14999    [area said, found nothing, gun violence, minut...
Name: ngrams-2, Length: 15000, dtype: object


In [ ]:
with open('docs.txt','w') as file:
    for i in docs:
        file.write(', '.join(i))
        file.write('\n')

In [ ]:
docs_copy = []
# Read docs from file to save memory
with open('docs.txt', 'r') as file:
    while True:
        line = file.readline().replace('\n', '')
        doc_ngrams = line.split(', ')
        docs_copy.append(doc_ngrams)
        
        if not line:
            break

In [18]:
binary_matrix = getBinaryMatrix(docs)

In [19]:
# Taking transpose so that the rows are shingles and columns are documents
binary_matrix = binary_matrix.T

In [20]:
print(binary_matrix.shape)
print(binary_matrix[:10])

(10000, 15000)
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]


## 4. We need hash function that maps integers 0, 1, . . . , k − 1 to bucket numbers 0 through k − 1. It might be impossible to avoid collisions but as long as the collions are too many it won't matter much. (10 points)
* The simplest would be using the builtin hash() function, it can be for example, hash(rownumber) % Numberofbuckets
* You can generate several of these hash functions by xoring a random integer (hash(rownumber)^randint) % Numberofbuckets
* It can also be a as simple as (rownumber * randint) % Numberofbuckets

In [ ]:
# def getHashFunctionValues(numrows, numhashfunctions):
#     # Your code Here
#     # return a matrix with hash values
#     # Each column represents a random permutation of row numbers
#     hash_matrix = []
#     for i in range(numrows):
#         temp = []
#         for j in range(numhashfunctions):
#             temp.append(np.random.randint(0, numrows + 1))
#         hash_matrix.append(temp)
#     return np.array(hash_matrix)

In [35]:
def getHashFunctionValues(numrows, numhashfunctions):
    # Your code Here
    # return a matrix with hash values
    hash_matrix = np.empty((numrows, numhashfunctions))
    for i in range(numrows):
        row = np.empty((1, numhashfunctions))
        for j in range(numhashfunctions):
            rand_int = np.random.randint(2^32-1)
            row[0, j] = hash(i) ^ rand_int % numrows
        hash_matrix[i] = row
    return hash_matrix

In [36]:
hash_matrix = getHashFunctionValues(binary_matrix.shape[0], num_hash_functions)

In [37]:
hash_matrix = hash_matrix.T

In [38]:
hash_matrix.shape

(200, 10000)

In [39]:
hash_matrix

array([[2.0000e+01, 9.0000e+00, 2.6000e+01, ..., 9.9900e+03, 1.0012e+04,
        9.9840e+03],
       [2.8000e+01, 2.7000e+01, 2.6000e+01, ..., 1.0001e+04, 1.0009e+04,
        9.9980e+03],
       [1.8000e+01, 1.8000e+01, 1.3000e+01, ..., 9.9980e+03, 1.0015e+04,
        9.9990e+03],
       ...,
       [2.3000e+01, 2.5000e+01, 1.6000e+01, ..., 1.0006e+04, 9.9860e+03,
        1.0011e+04],
       [3.0000e+00, 2.0000e+01, 9.0000e+00, ..., 9.9840e+03, 1.0002e+04,
        9.9920e+03],
       [1.1000e+01, 1.5000e+01, 7.0000e+00, ..., 1.0007e+04, 1.0006e+04,
        9.9910e+03]])

In [40]:
binary_matrix.shape

(10000, 15000)

## 5. Compute minhash following the faster algorithm from the lecture (10 points)

In [ ]:
# def getMinHashSignatureMatrix(binary_matrix, hash_val_matrix):
#     # return minhash signature matrix
#     signature_matrix = np.ones((binary_matrix.shape[1], hash_val_matrix.shape[0])) * np.inf
#     print(signature_matrix.shape)
#     for i in range(binary_matrix.shape[1]):
#         for j in range(hash_val_matrix.shape[0]):
#             index_comparison = (np.where(binary_matrix[:, i] == 1))[0]
#             if len(hash_val_matrix[j, index_comparison]) != 0:
#                 signature_matrix[i, j] = min(hash_val_matrix[j, index_comparison])
#     return signature_matrix

In [41]:
def getMinHashSignatureMatrix(binary_matrix, hash_val_matrix):
    # Each column represents one document's hash with 200 hash functions
    signature_matrix = np.ones((hash_val_matrix.shape[0], binary_matrix.shape[1])) * np.inf

    for i in range(binary_matrix.shape[0]):
        for j in np.where(binary_matrix[i] == 1)[0]: # for all docs containing the shingle
            for k in range(hash_val_matrix.shape[0]):
                if signature_matrix[k][j] > hash_val_matrix[k][i]:
                    signature_matrix[k][j] = hash_val_matrix[k][i]
    
    return signature_matrix

In [42]:
signature_matrix = getMinHashSignatureMatrix(binary_matrix, hash_matrix)

In [43]:
signature_matrix.shape

(200, 15000)

In [44]:
signature_matrix

array([[  5.,   1.,   8., ..., 425., 108., 107.],
       [  8.,   1.,  25., ..., 434.,  98., 112.],
       [  4.,   4.,  16., ..., 419., 123., 127.],
       ...,
       [  4.,   3.,   4., ..., 419., 126., 118.],
       [  3.,   0.,   0., ..., 427., 125., 114.],
       [  1.,   0.,  19., ..., 427., 108., 120.]])

## 6. Hash signature bands into buckets. Find a way to combine all the  signature values in a band and hash them into a number of buckets ususally very high. (10 points)
* Easiest way is to add all the signature values in the bucket and use a similar hash function like before
* You should use the same hash function for all bands. And all documents ending up in same bucket for at least one band are considered as candidate pairs.

In [45]:
def getLSH(signature_matrix, num_bands, num_buckets):
    # return lsh buckets or hash table
    lsh = {}
    rows_per_band = int(signature_matrix.shape[0] / num_bands)
    
    for i in range(num_bands):
        rand_int = np.random.randint(2 ^ 32 - 1)
        for j in range(signature_matrix.shape[1]):
            # Using the same hash function as before
            hash_val = hash(tuple(signature_matrix[i * rows_per_band:(i + 1) * rows_per_band, j])) ^ rand_int
            if hash_val not in lsh:
                lsh[hash_val] = set()
            lsh[hash_val].add(j)
    return lsh

In [46]:
lsh = getLSH(signature_matrix, num_bands, num_buckets)

## 7. Tune parameters to make sure the threshold is appropriate. (10 points)
* plot the probability of two similar items falling in same bucket for different threshold values

In [47]:
s = 0.7

## 8. Choose the best parameters and get nearest neighbors of each articles (20 points)
* Jaccard Similarity
* convert hash table into dictionary of article ids and its other articles that hashed in at least 1 same bucket

In [48]:
# Code from https://www.statology.org/jaccard-similarity-python/
def getJaccardSimilarityScore(C1, C2):
    intersection = len(list(set(C1).intersection(C2)))
    union = (len(C1) + len(C2)) - intersection
    return float(intersection) / union

## 9. Write the nearest neibhors of each document to submissions.csv (comma separated, first column is the current document followed by a list of nearest neighbors) file and get the score (10 points)

In [49]:
# convert hash table into dictionary of article ids and its other articles that hashed in at least 1 same bucket
nearest_neighbors = {}
for _, idx in lsh.items():
    for i in idx:
        if i not in nearest_neighbors:
            nearest_neighbors[i] = set()
        nearest_neighbors[i].update([x for x in idx if x != i])

In [50]:
n_copy = copy.deepcopy(nearest_neighbors)

In [ ]:
# Remove the neighbors in same buckets but have similarity score < threshold s
submission_id = []
submission_nid = []
scores = []
for article_id, neighbor_ids in n_copy.items():
    for nid in neighbor_ids:
        score = getJaccardSimilarityScore(binary_matrix[:,article_id], binary_matrix[:,nid])
        if score < s:
            nearest_neighbors[article_id].remove(nid) 
        else:
            # add to submission result
            submission_id.append(article_id)
            submission_nid.append(nid)
            scores.append(score)

In [ ]:
data = pd.DataFrame()
data['article_id'] = submission_id
data['neighbor_id'] = submission_nid
data['jaccard_score'] = scores
data.sort_values(by=['article_id', 'neighbor_id'], inplace=True)

In [ ]:
data.head(100)

In [ ]:
data.to_csv('submissions.csv')

## 10. Write a report + notebook + submission file in a zip file (5 points)